<a href="https://colab.research.google.com/github/Panperception/MML/blob/main/MML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialize

In [ ]:
!pip install tensorflow

## Learning on Many Manifolds

## Demo Code 0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

class SharedAutoencoder(nn.Module):
    def __init__(self, latent_dim, num_classes):
        super(SharedAutoencoder, self).__init__()

        # Shared encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*7*7, latent_dim)
        )

        # Separate latent space for each class (one per class)
        self.latent_spaces = nn.ModuleList([nn.Linear(latent_dim, latent_dim) for _ in range(num_classes)])

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64*7*7),
            nn.ReLU(),
            nn.Unflatten(1, (64, 7, 7)),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x, class_idx):
        # Shared encoder
        z = self.encoder(x)

        # Latent space transformation per class
        z_class = self.latent_spaces[class_idx](z)

        # Decoder
        reconstructed = self.decoder(z_class)
        return reconstructed, z_class

def train(model, dataloader, epochs=5, learning_rate=0.001):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data, targets in dataloader:
            optimizer.zero_grad()
            # Use the class index from the targets (it's already an integer)
            for i in range(data.size(0)):  # For each batch sample
                class_idx = targets[i].item()  # Convert target to int
                reconstructed, _ = model(data[i:i+1], class_idx)  # Use single image per forward pass
                loss = criterion(reconstructed, data[i:i+1])  # Reconstruction loss for the image
                loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}')

# Data Preparation (Using MNIST for simplicity)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Model Initialization
latent_dim = 128
num_classes = 10  # 10 classes for MNIST
model = SharedAutoencoder(latent_dim, num_classes)

# Training the Model
train(model, train_loader, epochs=5)

# Visualization of Reconstruction
def visualize_reconstruction(model, dataloader, class_idx=0):
    model.eval()
    data, targets = next(iter(dataloader))
    reconstructed, _ = model(data, class_idx)

    # Plot original and reconstructed images
    fig, axes = plt.subplots(2, 10, figsize=(15, 5))
    for i in range(10):
        axes[0, i].imshow(data[i].squeeze().cpu().numpy(), cmap='gray')
        axes[1, i].imshow(reconstructed[i].squeeze().cpu().detach().numpy(), cmap='gray')
        axes[0, i].axis('off')
        axes[1, i].axis('off')

    plt.show()

# Visualize reconstruction for a specific class
visualize_reconstruction(model, train_loader, class_idx=0)  # For class 0 (e.g., '0' digit in MNIST)


## Demo Code 1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------
# Model Components
# -----------------------

class SharedEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU()
        )

    def forward(self, x):
        return self.shared(x)

class LatentBranch(nn.Module):
    def __init__(self, latent_dim=16):
        super().__init__()
        self.fc = nn.Linear(128, latent_dim)

    def forward(self, x):
        return self.fc(x)

class SharedDecoder(nn.Module):
    def __init__(self, latent_dim=16):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, z):
        return self.decoder(z).view(-1, 1, 28, 28)

class MultiLatentAutoencoder(nn.Module):
    def __init__(self, latent_dim=16, num_classes=10):
        super().__init__()
        self.encoder = SharedEncoder()
        self.latent_branches = nn.ModuleList([LatentBranch(latent_dim) for _ in range(num_classes)])
        self.decoder = SharedDecoder(latent_dim)

    def forward(self, x, labels):
        shared = self.encoder(x)
        latents = torch.stack([branch(shared) for branch in self.latent_branches], dim=1)  # [B, C, latent_dim]
        z = latents[torch.arange(x.size(0)), labels]  # Select class-specific latent
        out = self.decoder(z)
        return out

# -----------------------
# Dataset
# -----------------------

transform = transforms.ToTensor()

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# -----------------------
# Training Setup
# -----------------------

model = MultiLatentAutoencoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# -----------------------
# Training Loop
# -----------------------

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images, labels)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# -----------------------
# Evaluation / Visualization
# -----------------------

model.eval()
images, labels = next(iter(test_loader))
images, labels = images.to(device), labels.to(device)

with torch.no_grad():
    recon = model(images, labels)

# Show original and reconstructed
def show_images(original, reconstructed, n=10):
    plt.figure(figsize=(15, 3))
    for i in range(n):
        # Original
        plt.subplot(2, n, i + 1)
        plt.imshow(original[i].cpu().squeeze(), cmap='gray')
        plt.axis('off')
        # Reconstructed
        plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed[i].cpu().squeeze(), cmap='gray')
        plt.axis('off')
    plt.suptitle("Top: Original | Bottom: Reconstructed", fontsize=14)
    plt.show()

show_images(images, recon)


## Demo Code 2
### Key Takeaways
* Each class's decoder models a distinct digit manifold.
* RandomAffine introduces group action (SO(2), translation), challenging the model to be invariant or equivariant.
* Decoder selection requires label knowledge; removing this label makes inference NP-hard (you’d need to guess decoder + latent code jointly).
* This aligns with the theoretical proof sketch — combinatorial search across submanifolds under transformations.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# --- 1. Data Loading with Group Action (rotation) ---
transform_train = transforms.Compose([
    transforms.RandomAffine(degrees=30),  # simulates group action SO(2)
    transforms.ToTensor()
])
transform_test = transforms.ToTensor()

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

# --- 2. Shared Encoder (learns common latent structure) ---
class Encoder(nn.Module):
    def __init__(self, latent_dim=32):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),  # 28 -> 14
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # 14 -> 7
            nn.ReLU()
        )
        self.fc = nn.Linear(32 * 7 * 7, latent_dim)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# --- 3. Class-Specific Decoder (models separate submanifolds) ---
class Decoder(nn.Module):
    def __init__(self, latent_dim=32):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 32 * 7 * 7)
        self.deconv = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),  # 7 -> 14
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),   # 14 -> 28
            nn.Sigmoid()
        )

    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 32, 7, 7)
        return self.deconv(z)

# --- 4. Full Model with Shared Encoder + Per-Class Decoders ---
class ManifoldAutoencoder(nn.Module):
    def __init__(self, latent_dim=32, num_classes=10):
        super().__init__()
        self.encoder = Encoder(latent_dim)
        self.decoders = nn.ModuleList([Decoder(latent_dim) for _ in range(num_classes)])

    def forward(self, x, labels):
        z = self.encoder(x)
        recon = torch.zeros_like(x)
        for i in range(x.size(0)):
            recon[i] = self.decoders[labels[i]](z[i].unsqueeze(0))
        return recon

# --- 5. Training the Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ManifoldAutoencoder(latent_dim=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

def train(model, loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            x_hat = model(x, y)
            loss = loss_fn(x_hat, x)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} | Avg Loss: {total_loss / len(loader):.4f}")

train(model, train_loader, epochs=5)

# --- 6. Visualize Reconstructions ---
def visualize_reconstruction(model, loader):
    model.eval()
    x, y = next(iter(loader))
    x, y = x.to(device), y.to(device)
    with torch.no_grad():
        x_hat = model(x, y)
    x, x_hat = x.cpu(), x_hat.cpu()

    fig, axes = plt.subplots(2, 10, figsize=(12, 3))
    for i in range(10):
        axes[0, i].imshow(x[i][0], cmap='gray')
        axes[1, i].imshow(x_hat[i][0], cmap='gray')
        axes[0, i].axis('off')
        axes[1, i].axis('off')
    plt.suptitle("Top: Original | Bottom: Reconstructed via Class-Specific Decoder")
    plt.show()

visualize_reconstruction(model, test_loader)


## Demo Code 3

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MNIST data loader
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Shared Encoder
class Encoder(nn.Module):
    def __init__(self, latent_dim=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )

    def forward(self, x):
        return self.net(x)

# Class-specific Decoder
class Decoder(nn.Module):
    def __init__(self, latent_dim=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 28*28),
            nn.Sigmoid()
        )

    def forward(self, z):
        x_hat = self.net(z)
        return x_hat.view(-1, 1, 28, 28)

# Full Autoencoder
class MultiDecoderAutoencoder(nn.Module):
    def __init__(self, num_classes=10, latent_dim=32):
        super().__init__()
        self.encoder = Encoder(latent_dim)
        self.decoders = nn.ModuleList([Decoder(latent_dim) for _ in range(num_classes)])

    def forward(self, x, label):
        z = self.encoder(x)
        x_hat = torch.zeros_like(x)
        for i in range(len(self.decoders)):
            mask = (label == i)
            if mask.any():
                x_hat[mask] = self.decoders[i](z[mask])
        return x_hat

model = MultiDecoderAutoencoder().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Training
def train_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        x_hat = model(x, y)
        loss = criterion(x_hat, x)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    print(f"[Train Loss] {total_loss / len(loader.dataset):.4f}")

# Train the model
for epoch in range(1, 6):
    print(f"Epoch {epoch}")
    train_epoch(model, train_loader, optimizer)

# Unsupervised decoder inference
def infer_decoder_unsupervised(model, x_batch):
    model.eval()
    x_batch = x_batch.to(device)
    with torch.no_grad():
        z = model.encoder(x_batch)
        errors = []
        for decoder in model.decoders:
            x_hat = decoder(z)
            err = F.mse_loss(x_hat, x_batch, reduction='none')
            err = err.view(err.size(0), -1).mean(dim=1)
            errors.append(err.unsqueeze(1))
        all_errors = torch.cat(errors, dim=1)
        best_decoder = all_errors.argmin(dim=1)
    return best_decoder.cpu()

# Evaluate unsupervised accuracy
def evaluate_unsupervised(model, loader):
    correct = 0
    total = 0
    for x, y in loader:
        pred = infer_decoder_unsupervised(model, x)
        correct += (pred == y).sum().item()
        total += x.size(0)
    print(f"[Unsupervised Inference Accuracy] {correct}/{total} = {correct/total:.2%}")

evaluate_unsupervised(model, test_loader)


## XAI

In [ ]:
!pip install shap
import shap

In [ ]:
xai_model = model_dmlp_adam

In [ ]:
img_rows=32
img_cols=32
background = x_train[np.random.choice(x_train.shape[0], 500, replace=False)]

In [ ]:
e = shap.DeepExplainer(xai_model, background)

In [ ]:
background = x_train[np.random.choice(x_train.shape[0], 500, replace=False)]
e = shap.DeepExplainer(xai_model, background)
shap_values = e.shap_values(x_test[1:5])
shap.image_plot(shap_values, -x_test)



In [ ]:
s_values = [(np.reshape(x,(x.shape[0],img_rows,img_cols))) for x in shap_values]
x_tst = np.reshape(x_test[1:5],(x_test[1:5].shape[0],img_rows,img_cols))
shap.image_plot(s_values, -x_tst)


In [ ]:
from tensorflow.python.keras.losses import categorical_crossentropy
from cxplain import MLPModelBuilder, ZeroMasking, CXPlain

x_train, y_train, x_test = ....  # Your dataset
explained_model = ...    # The model you wish to explain.

# Define the model you want to use to explain your __explained_model__.
# Here, we use a neural explanation model with a
# multilayer perceptron (MLP) architecture.
model_builder = MLPModelBuilder(num_layers=2, num_units=64, batch_size=256, learning_rate=0.001)

# Define your masking operation - the method of simulating the
# removal of input features used internally by CXPlain - ZeroMasking is typically a sensible default choice for tabular and image data.
masking_operation = ZeroMasking()

# Define the loss with which each input features' associated reduction in prediction error is calculated.
loss = categorical_crossentropy

# Build and fit a CXPlain instance.
explainer = CXPlain(explained_model, model_builder, masking_operation, loss)
explainer.fit(x_train, y_train)

# Use the __explainer__ to obtain explanations for the predictions of your __explained_model__.
attributions = explainer.explain(x_test)

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import numpy as np

# Overall F1 score
y_pred=np.argmax(model_dmlp_adam.predict(x_test), axis=1)
print("F1 Score:  ", f1_score(y_test, y_pred, average="macro"))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall:    ", recall_score(y_test, y_pred, average="macro"))
print("Accuracy:    ", accuracy_score(y_test, y_pred))

# Overall F1 score

#intermediate_layer_model = keras.Model(inputs=model_dmlp_SGD.input,
#                                 outputs=model_dmlp_SGD.layers[0].output)
#intermediate_output = intermediate_layer_model.predict(x_test)
#ws=model_dmlp_SGD.layers[0].get_weights()
#y00=np.matmul(x_test,ws[0])+np.tile(ws[1],[x_test.shape[0],1])


y_pred=np.argmax(model_dmlp_SGD.predict(x_test), axis=1)
print("F1 Score:  ", f1_score(y_test, y_pred, average="macro"))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall:    ", recall_score(y_test, y_pred, average="macro"))
print("Accuracy:    ", accuracy_score(y_test, y_pred))
#print("Accuracy:    ", np.mean(y_pred.reshape(1171,1)==y_test))

## Pretrained VGG16

In [ ]:
import keras
import matplotlib.pyplot as plt

#img_size_target = 32;
#img_input = tf.keras.layers.Input(shape=(img_size_target, img_size_target,1))
#img_conc = tf.keras.layers.Concatenate()([img_input, img_input, img_input])

from keras.applications.vgg16 import VGG16

## Loading VGG16 model
pre_model = VGG16(include_top=False, weights='imagenet', input_shape=(32,32,3))
pre_model.trainable = False ## Not trainable weights

from keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(32, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')

model = models.Sequential([
    pre_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

pre_model.summary()
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)


In [ ]:
one_hot_labels = keras.utils.np_utils.to_categorical(y_train, num_classes=2)
hist_v16=model.fit(np.repeat(x_train.reshape((x_train.shape[0],32,32,1)), 3, axis=3), one_hot_labels, epochs=100, validation_split=0.2, batch_size=32, callbacks=[])

plt.plot(hist_v16.history["loss"]);
plt.xlabel('Epochs');
plt.ylabel('Training Error');


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import numpy as np

# Overall F1 score
y_pred=np.argmax(model.predict(np.repeat(x_test.reshape((x_test.shape[0],32,32,1)), 3, axis=3)), axis=1)
print("F1 Score:  ", f1_score(y_test, y_pred, average="macro"))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall:    ", recall_score(y_test, y_pred, average="macro"))
print("Accuracy:    ", accuracy_score(y_test, y_pred))
print("Accuracy:    ", np.mean(y_pred.reshape(1171,1)==y_test))



###XAI Demo of VGG16 Model

In [ ]:
xai_model = model

In [ ]:
background = x_trn[np.random.choice(x_trn.shape[0], 500, replace=False)]
e = shap.DeepExplainer(xai_model, background)

In [ ]:
x_sel=x_tst[np.random.choice(x_trn.shape[0], 5, replace=False)]
shap_values = e.shap_values(x_sel)
shap.image_plot(shap_values, x_sel)

In [ ]:
y_pred=np.argmax(model.predict(x_sel), axis=1)


## Pretrained ResNet50

In [ ]:
import keras
import matplotlib.pyplot as plt

#img_size_target = 32;
#img_input = tf.keras.layers.Input(shape=(img_size_target, img_size_target,1))
#img_conc = tf.keras.layers.Concatenate()([img_input, img_input, img_input])

from keras.applications.resnet import ResNet50

## Loading VGG16 model
pre_model = ResNet50(include_top=False, weights='imagenet', input_shape=(32,32,3))
pre_model.trainable = True ## Not trainable weights

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(32, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')

model = models.Sequential([
    pre_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

pre_model.summary()
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)


In [ ]:
one_hot_labels = keras.utils.np_utils.to_categorical(y_train, num_classes=2)
hist_ResNet=model.fit(np.repeat(x_train.reshape((x_train.shape[0],32,32,1)), 3, axis=3), one_hot_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[])


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import numpy as np

# Overall F1 score
y_pred=np.argmax(model.predict(np.repeat(x_test.reshape((x_test.shape[0],32,32,1)), 3, axis=3)), axis=1)
print("F1 Score:  ", f1_score(y_test, y_pred, average="macro"))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall:    ", recall_score(y_test, y_pred, average="macro"))
print("Accuracy:    ", accuracy_score(y_test, y_pred))
print("Accuracy:    ", np.mean(y_pred.reshape(1171,1)==y_test))



## Pretrained VGG19

In [ ]:
import keras
import matplotlib.pyplot as plt

#img_size_target = 32;
#img_input = tf.keras.layers.Input(shape=(img_size_target, img_size_target,1))
#img_conc = tf.keras.layers.Concatenate()([img_input, img_input, img_input])

from keras.applications.vgg19 import VGG19

## Loading VGG16 model
pre_model = VGG19(include_top=False, weights='imagenet', input_shape=(32,32,3))
pre_model.trainable = False ## Not trainable weights

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(32, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')

model = models.Sequential([
    pre_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

pre_model.summary()
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)


In [ ]:
one_hot_labels = keras.utils.np_utils.to_categorical(y_train, num_classes=2)
hist_v19=model.fit(np.repeat(x_train.reshape((x_train.shape[0],32,32,1)), 3, axis=3), one_hot_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[])


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import numpy as np

# Overall F1 score
y_pred=np.argmax(model.predict(np.repeat(x_test.reshape((x_test.shape[0],32,32,1)), 3, axis=3)), axis=1)
print("F1 Score:  ", f1_score(y_test, y_pred, average="macro"))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall:    ", recall_score(y_test, y_pred, average="macro"))
print("Accuracy:    ", accuracy_score(y_test, y_pred))
print("Accuracy:    ", np.mean(y_pred.reshape(1171,1)==y_test))


## Pretrained Inception

In [ ]:
import keras
import matplotlib.pyplot as plt

#img_size_target = 32;
#img_input = tf.keras.layers.Input(shape=(img_size_target, img_size_target,1))
#img_conc = tf.keras.layers.Concatenate()([img_input, img_input, img_input])

from keras.applications import Xception

## Loading Xception model
pre_model = Xception(weights='imagenet',
                              include_top=False,
                              input_shape=(256, 256, 3))
pre_model.trainable = False ## Not trainable weights

from keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(32, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')

model = models.Sequential([
    pre_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

pre_model.summary()
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
one_hot_labels = keras.utils.np_utils.to_categorical(y_train, num_classes=2)

hist_Inception=model.fit(np.repeat(x_train.reshape((x_train.shape[0],32,32,1)), 3, axis=3).resize(256,256), one_hot_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[])


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import numpy as np

# Overall F1 score
y_pred=np.argmax(model.predict(np.repeat(x_test.reshape((x_test.shape[0],32,32,1)), 3, axis=3)), axis=1)
print("F1 Score:  ", f1_score(y_test, y_pred, average="macro"))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall:    ", recall_score(y_test, y_pred, average="macro"))
print("Accuracy:    ", accuracy_score(y_test, y_pred))
print("Accuracy:    ", np.mean(y_pred.reshape(1171,1)==y_test))


## Summary

In [ ]:
#plt.plot(hist1.history['loss']);
plt.plot(hist_v16.history['loss']);
plt.plot(hist_v19.history['loss']);
plt.plot(hist_ResNet.history['loss']);
plt.legend(['V16', 'V19', 'ResNet50'])

#plt.plot(hist_ResNet.history['accuracy']);
#plt.plot(hist_v19.history['val_loss']);
#plt.plot(hist_v19.history['val_accuracy']);
plt.xlabel('Epochs');
plt.ylabel('Training Criteria');